### Scraping Forbes 2000 Companies list

In [12]:
!pip install websocket-client

In [4]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import pandas as pd
import time

# Setup driver
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
service = Service()  # You can pass path to chromedriver if needed
driver = webdriver.Chrome(service=service, options=options)

# Load Forbes page
driver.get("https://www.forbes.com/lists/global2000/")
time.sleep(5)  # Let JS load

# Select 'United States' from dropdown
select = Select(driver.find_element("id", "dropdown"))
select.select_by_visible_text("United States")
time.sleep(5)  # Let companies load

# Get page content and parse
soup = BeautifulSoup(driver.page_source, "html.parser")
companies = soup.select("div.row-cell-value.nameField")
company_names = [tag.get_text(strip=True) for tag in companies]

# Save to CSV
df = pd.DataFrame(company_names, columns=["Company Name"])
df.to_csv("forbes_us_companies.csv", index=False)

print("✅ Done scraping companies from the United States!")

driver.quit()


✅ Done scraping companies from the United States!


In [15]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
import time

options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
service = Service()
driver = webdriver.Chrome(service=service, options=options)

driver.get("https://www.forbes.com/lists/global2000/")
time.sleep(5)  # Wait for initial page load

# Select United States
select = Select(driver.find_element(By.ID, "dropdown"))
select.select_by_visible_text("United States")
time.sleep(5)  # Let companies start loading

# Scroll to bottom repeatedly to load all companies
SCROLL_PAUSE = 2
last_height = driver.execute_script("return document.body.scrollHeight")
new_companies_loaded = True

while new_companies_loaded:
    # Scroll down
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(SCROLL_PAUSE)
    # Check if new height (new data loaded)
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        new_companies_loaded = False
    else:
        last_height = new_height

# After all records loaded, parse
soup = BeautifulSoup(driver.page_source, "html.parser")
companies = soup.select("div.row-cell-value.nameField")
company_names = [tag.get_text(strip=True) for tag in companies]

driver.quit()

# Save to CSV
df = pd.DataFrame(company_names, columns=["Company Name"])
df.to_csv("forbes_us_companies_full.csv", index=False)
print(f"✅ Scraped {len(company_names)} companies and saved to 'forbes_us_companies_full.csv'")


✅ Scraped 621 companies and saved to 'forbes_us_companies_full.csv'
